**Mount drive and clone repository**

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

# Clone repository
import sys
!git clone https://github.com/fgiacome/MLDL23-FL-project.git
sys.path.append('./MLDL23-FL-project/')
%cd MLDL23-FL-project

In [ ]:
!git checkout self_training

**Transforms and model importations**

In [ ]:
# Importations
import main
import torch
import random
from datasets.idda import IDDADataset
from models import deeplabv3, mobilenetv2

# Fix random seed
random.seed(300890)

# args for dataset and model importation
class args:
  dataset = 'idda'
  model = 'deeplabv3_mobilenetv2'

# Model
model = main.model_init(args).cuda()

# Transforms
train_transforms, test_transforms = main.get_transforms(args)

**Train, Test Same Dom, Test Diff Dom importations**

In [ ]:
# Importations
import os
import json

# Initialization of clients datasets
root = 'data/idda'
with open(os.path.join(root, 'train.json'), 'r') as f:
  clients_dict = json.load(f)

# Dict -> datasets
train_clients_df = dict()
for client_id in clients_dict.keys():
  train_clients_df[client_id] = IDDADataset(root = root, list_samples = clients_dict[client_id],
                                             transform = train_transforms, client_name = client_id)
train_clients_names = list(train_clients_df.keys())

# Test set
with open(os.path.join(root, 'test_same_dom.txt'), 'r') as f:
    test_same_dom_data = f.read().splitlines()
    test_same_dom_df = IDDADataset(root = root, list_samples = test_same_dom_data,
                                        transform = test_transforms,
                                        client_name='test_same_dom')

with open(os.path.join(root, 'test_diff_dom.txt'), 'r') as f:
    test_diff_dom_data = f.read().splitlines()
    test_diff_dom_df = IDDADataset(root=root, list_samples = test_diff_dom_data,
                                        transform = test_transforms,
                                        client_name='test_diff_dom')

**Combinations**

In [ ]:
# Combinations to test with the format [n_clients_round, n_local_epochs, n_rounds_before_teacher_update (0: only once)]
combinations = {
                'comb_2_3_0': [2, 3, 0],
                'comb_2_3_1': [2, 3, 1],
                'comb_2_3_3': [2, 3, 3],
                'comb_4_6_0': [4, 6, 0],
                ## Continua da qui ##
                'comb_4_6_1': [4, 6, 1],
                'comb_4_6_5': [4, 6, 5],
                'comb_4_6_10': [4,6,10]
              }

In [ ]:
def load_best_model_weights_from_point_3():
  """
  This function returns the weight dictionary of the best
  model from the previous point, ready for loading on the clients
  and teachers via PyTorch's load_state_dict
  """
  ##path = "/content/drive/MyDrive/mldl-project-2b/FDA_P1_Models_Checkpoints/CP_lr1e3_T/checkpoint_e15.json"
  ## nuovo path
  ## SCOMMENTANDO IL PATH, SCOMMENTARE ANCHE IL NUOVO FILENAME E COMMENTARE IL VECCHIO
  path = "/content/drive/MyDrive/mldl-project-2b/FDA_P2_Models_Checkpoints/CP_beta6e3/checkpoint_e15.json"
  ##
  state_dict_dict = torch.load(path)
  return state_dict_dict["model_state_dict"]

**Test cell**

In [ ]:
# Client and Server importationsù
import copy
from client import Client
from server import Server

# Select combination
################################################################################
############################## CHOOSE COMB! ####################################
################################################################################
comb = 'comb_4_6_10' ###########################################################
################################################################################
############################## CHOOSE COMB! ####################################
################################################################################
################################################################################
# Comb_dict formats: [<n_clients_round>, <n_local_epochs>] #####################
#################### [       0     ,             1       ] #####################
################################################################################

# Assign combinations
clients_per_round = combinations[comb][0]
epochs_per_round = combinations[comb][1]
teacher_update_rounds = combinations[comb][2]

# Number of total rounds
num_rounds = 50

# !!CHOOSE REDUCTION!!
# reduction \in \{'MeanReduction, HardNegativeMining'\}
reduction = 'MeanReduction'

# Clients initializations
# - Train clients
train_clients = [Client(client_dataset = train_clients_df[name], batch_size = 8,
                 model = main.model_init(args).cuda(), dataloader = 'train',
                 optimizer = 'Adam', lr = 1e-3, device = 'cuda:0',
                 reduction = reduction, self_training = True) for name in train_clients_names]

# initialize the teachers
for client in train_clients:
  teacher_model = main.model_init(args).cuda()
  client.set_teacher(model)

# - Test clients [same_dom, diff_dom]
test_clients = [Client(client_dataset = test_same_dom_df, batch_size = 16,
                model = main.model_init(args).cuda(), dataloader = 'test',
                device = 'cuda:0', reduction = reduction),
                Client(client_dataset = test_diff_dom_df, batch_size = 16,
                model = main.model_init(args).cuda(), dataloader = 'test',
                device = 'cuda:0', reduction = reduction)]

# Server initialization
weights_dict = load_best_model_weights_from_point_3()
server_model = main.model_init(args).cuda()
server_model.load_state_dict(weights_dict)
server = Server(clients_per_round = clients_per_round, num_rounds = num_rounds,
                epochs_per_round = epochs_per_round, train_clients = train_clients,
                test_clients = test_clients, model = server_model,
                metrics = None, random_state = 300890, teacher_update_rounds = teacher_update_rounds)

# Train server on selected combination
results = server.train(path="/content/drive/MyDrive/mldl-project-2b/CheckpointsLongTraining/checkpoint_round")
results_eval = server.eval_train()
results_test = server.test()

# Cast dict
# Results dict format {<round_index>: <results>}
results_dict = {round_idx: results[round_idx] for round_idx in range(num_rounds)}
results_complete = {
      "Train" : results_dict,
      "Eval" : results_eval,
      "Test" : results_test
  }

# Save results
# Define file name
#filename = 'results_' + comb + '.json'
filename = 'FDA_results_long_' + comb + '.json'
path = '/content/drive/MyDrive/mldl-project-2b/Results4/'

# Save data on file system (Remember to download it!)
with open(path + filename, 'w') as fp:
  json.dump(results_complete, fp)

**Save results**